# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [4]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect("data.sqlite")

In [7]:
pd.read_sql("""
    select * from offices

""", conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [10]:
# Your code here
pd.read_sql("""
    select employees.firstName, employees.lastName from employees
    join offices on employees.officeCode = offices.officeCode
    where offices.city = 'Boston'
;
""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [13]:
# Your code here
pd.read_sql("""
    select offices.officeCode, offices.city, count(employees.employeeNumber) as num_employees from offices
    join employees on offices.officeCode = employees.officeCode
    group by offices.officeCode
;
""", conn)

,officeCode,city,num_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [15]:
"""
Question 1
How many customers are from the same state in the US? 
"""
pd.read_sql("""
    select state, count(*) as customers_per_state
    from customers
    where country = 'USA'
    group by state
;
""", conn)


# Your code here

,state,customers_per_state
0,CA,11
1,CT,4
2,MA,9
3,NH,1
4,NJ,1
5,NV,1
6,NY,6
7,PA,3


In [17]:
"""
Question 2
What are the highest 3 order payments and their associated customer names? 
"""

pd.read_sql("""
    select customers.customerName, payments.amount from customers
    join payments on customers.customerNumber = payments.customerNumber
    order by payments.amount desc
    limit 3
;
""", conn)

# Your code here

,customerName,amount
0,Euro+ Shopping Channel,120166.58
1,Euro+ Shopping Channel,116208.40
2,Mini Gifts Distributors Ltd.,111654.40


In [20]:
"""
Question 3
How many payments had values of less than $20000?
"""

pd.read_sql("""
    select count(*) from payments
    where amount < 20000
;
""", conn)

# Your code here

,count(*)
0,78


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [24]:
# Your code here

pd.read_sql("""
    select employees.employeeNumber, employees.lastName, employees.firstName, products.productName
    from employees
    join customers on employees.employeeNumber = customers.salesRepEmployeeNumber
    join orders on customers.customerNumber = orders.customerNumber
    join orderdetails on orders.orderNumber = orderdetails.orderNumber
    join products on products.productCode = orderdetails.productCode
;
""", conn)


,employeeNumber,lastName,firstName,productName
0,1165,Jennings,Leslie,1958 Setra Bus
1,1165,Jennings,Leslie,1940 Ford Pickup Truck
2,1165,Jennings,Leslie,1939 Cadillac Limousine
3,1165,Jennings,Leslie,1996 Peterbilt 379 Stake Bed with Outrigger
4,1165,Jennings,Leslie,1968 Ford Mustang
...,...,...,...,...
2991,1702,Gerard,Martin,1954 Greyhound Scenicruiser
2992,1702,Gerard,Martin,1950's Chicago Surface Lines Streetcar
2993,1702,Gerard,Martin,Diamond T620 Semi-Skirted Tanker
2994,1702,Gerard,Martin,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [25]:
# Your code here
pd.read_sql("""
    select employees.employeeNumber, employees.lastName, employees.firstName, count(products.productName) as quantityTotal
    from employees
    join customers on employees.employeeNumber = customers.salesRepEmployeeNumber
    join orders on customers.customerNumber = orders.customerNumber
    join orderdetails on orders.orderNumber = orderdetails.orderNumber
    join products on products.productCode = orderdetails.productCode
    group by employees.employeeNumber
;
""", conn)

,employeeNumber,lastName,firstName,quantityTotal
0,1165,Jennings,Leslie,331
1,1166,Thompson,Leslie,114
2,1188,Firrelli,Julie,124
3,1216,Patterson,Steve,152
4,1286,Tseng,Foon Yue,142
5,1323,Vanauf,George,211
6,1337,Bondur,Loui,177
7,1370,Hernandez,Gerard,396
8,1401,Castillo,Pamela,272
9,1501,Bott,Larry,236


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [26]:
# Your code here
pd.read_sql("""
    select employees.employeeNumber, employees.lastName, employees.firstName, count(products.productName) as quantityTotal
    from employees
    join customers on employees.employeeNumber = customers.salesRepEmployeeNumber
    join orders on customers.customerNumber = orders.customerNumber
    join orderdetails on orders.orderNumber = orderdetails.orderNumber
    join products on products.productCode = orderdetails.productCode
    group by employees.employeeNumber
    having quantityTotal > 200
;
""", conn)

,employeeNumber,lastName,firstName,quantityTotal
0,1165,Jennings,Leslie,331
1,1323,Vanauf,George,211
2,1370,Hernandez,Gerard,396
3,1401,Castillo,Pamela,272
4,1501,Bott,Larry,236
5,1504,Jones,Barry,220


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!